# 1. Setup

In [1]:
import sys
sys.path.append('../..')
from mobiquity.names import *

from pqdm.processes import pqdm
from scipy.spatial import KDTree

# 2. Compute access

## 2.1. Load data

### 2.1.1. Zone boundaries

In [2]:
zone_msa = U.load(DATA / 'zones/zones_2020.parquet', geom=False,
                  columns='geoid scale state msa aland'.split(),
                  filters=[('top_msa', '==', True)]).disp()

144,489 rows x 5 cols; Memory: 10.4 MiB


,geoid,scale,state,msa,aland
,<object>,<category>,<category>,<category>,<float32>
0,04013,County,AZ,Phoenix,9201.749023


In [3]:
zone_in = U.load(DATA / 'zones/in_2020.parquet', geom=False,
                 columns=['geoid', 'scale', 'county', 'aland'],
                 filters=[('aland', '>', 0)]).disp()

7,072 rows x 4 cols; Memory: 0.7 MiB


,geoid,scale,county,aland
,<object>,<category>,<category>,<float32>
0,18001,County,Adams,338.924896


### 2.1.2. Population

In [4]:
popu = U.load(DATA / 'ses/acs/acs_2021.parquet',
              columns=['geoid', 'popu', 'labor']).disp()

322,202 rows x 3 cols; Memory: 26.0 MiB


,geoid,popu,labor
,<object>,<float64>,<float64>
0,01001,58239.0,27550.0


### 2.1.3. Opportunities

In [5]:
opport = U.load(DATA / 'access/opport/opportunities.parquet').disp()

2,253,050 rows x 5 cols; Memory: 52.9 MiB


,geoid,scale,purpose,kind,opport
,<category>,<category>,<category>,<category>,<int32>
0,01001,County,Work,All,12318


### 2.1.4. Impedance function

In [6]:
imped_params = U.load(DATA / 'impedance_params_double.csv').disp(None)

8 rows x 5 cols; Memory: 0.0 MiB


,mode,purpose,β1,β2,r2
,<object>,<object>,<float64>,<float64>,<float64>
0,Drive,Work,-0.007127,1.501453,0.998159
1,Drive,Non-work,-0.020097,1.36163,0.995175
2,Transit,Work,-0.000166,2.096192,0.999184
3,Transit,Non-work,-0.002062,1.608027,0.998069
4,Walk,Work,-0.053588,1.085346,0.997923
5,Walk,Non-work,-0.058269,1.007449,0.997875
6,Bike,Work,-0.007653,1.591415,0.994467
7,Bike,Non-work,-0.02808,1.145222,0.991943


## 2.2. Travel times

### 2.2.1. Indiana (GDM)

In [7]:
tt_in = U.load(DATA / 'access/distances/gdm_2020.parquet',
               filters=[('time', '<=', 90 * 60)],
               columns='scale mode src trg time'.split())
tt_in.time /= 60 # convert to minutes
same_zone = (tt_in[['scale', 'mode', 'src']].drop_duplicates()
             .assign(trg=lambda df: df.src, time=0.))
tt_in = pd.concat([tt_in, same_zone]).astype(
    D(scale=CAT, mode=CAT, src=CAT, trg=CAT, time=F32)).disp()

8,578,746 rows x 5 cols; Memory: 148.7 MiB


,scale,mode,src,trg,time
,<category>,<category>,<category>,<category>,<float32>
0,Tract,Transit,18105001304,18105001301,54.466667


### 2.2.2. Top 50 MSAs (OSRM)

In [8]:
def get_tt_msa(msa):
    res = []
    cols = D(src_geoid='src', trg_geoid='trg', duration='time')
    for mode in ['drive', 'bike', 'walk']:
        fpath = DATA / f'access/distances/osrm/msa/{msa}/bg_{mode}_2020.parquet'
        df = pd.read_parquet(fpath, columns=list(cols))
        df.duration /= 60 # convert to minutes
        res.append(df.rename(columns=cols).assign(mode=mode.title()))
    df = pd.concat(res).reset_index(drop=1)#.astype(D(mode=CAT))
    res = [df.assign(scale='BG')]
    for scale, nchar in D(County=5, Tract=11).items():
        d = df.assign(src=df.src.str[:nchar], trg=df.trg.str[:nchar])
        d = d.groupby(['src', 'trg', 'mode'])['time'].agg(np.median)
        res.append(d.reset_index().assign(scale=scale))
    df = pd.concat(res).reset_index(drop=1)
    cat_cols = ['scale', 'mode', 'src', 'trg']
    df = df.astype({x: CAT for x in cat_cols})[cat_cols + ['time']]
    return df

df = get_tt_msa('austin').disp()

2,537,638 rows x 5 cols; Memory: 24.6 MiB


,scale,mode,src,trg,time
,<category>,<category>,<category>,<category>,<float32>
0,BG,Drive,480219501011,480219501011,0.0


## 2.3. Compute accessibility

- Impedance weight: $$ w_{i,j}^{k,m} = f_{k,m}(c_{m,i,j}) = c_{m,i,j}^{\alpha_{k,m}} e^{\beta_{k,m} c_{m,i,j}} $$
- Contour: $$ A_i^{k,m,t} = \sum_{j\in C_i(t)} o_j^k $$
- Gravity: $$ A_i^{k,m,t} = \sum_{j\in C_i(t)} o_j^k\cdot w_{i,j}^{k,m} $$
- Competition: $$ A_i^{k,m,t} = \sum_{j\in C_i(t)} \frac{o_j^k\cdot w_{i,j}^{k,m}}{\sum_{l\in C_j(t)} p_j^k\cdot w_{i,j}^{k,m}} $$

In [9]:
thresholds = (15, 30, 45, 60, 90) # travel time thresholds (minutes)

In [10]:
def get_access(fname, times, opport=opport, popu=popu, params=imped_params,
               thresholds=thresholds, overwrite=False):
    outpath = DATA / f'access/access/{fname}.parquet'
    if (df := U.checkfile(outpath, overwrite)) is not None: return df
    res = []
    times = times.astype(D(src=str, trg=str))
    opport_grps = opport.groupby(['kind', 'purpose'])
    for thresh in tqdm(thresholds):
        tt = times[times.time <= thresh].copy()
        for (scale, mode), od_ in tt.groupby(['scale', 'mode']):
            for (kind, purp), df in opport_grps:
                df = df[df['scale'] == scale]
                r = U.filt(params, mode=mode, purpose=purp).iloc[0]
                od = od_[['src', 'trg', 'time']].copy()
                od['weight'] = np.exp(r['β1'] * od.pop('time') ** r['β2'])
                od = od.merge(df.rename(columns=D(geoid='trg'))[['trg', 'opport']])
                od['numer'] = od['opport'] * od['weight']
                xs = (od.groupby('src')[['opport', 'numer']].sum()
                      .rename(columns=D(opport='Contour', numer='Gravity'))
                      .astype(F32).rename_axis('geoid').reset_index()
                      .melt('geoid', var_name='measure', value_name='access'))
                fca = od.merge(popu, left_on='src', right_on='geoid')
                dmd_col = 'labor' if purp == 'Work' else 'popu'
                fca['denom'] = fca[dmd_col] * fca['weight']
                denom = fca.groupby('trg')['denom'].sum().reset_index()
                od = od.merge(denom, on='trg')
                od['ratio'] = od['numer'] / od['denom']
                fca = (od.groupby('src')['ratio'].sum().rename('access')
                       .astype(F32).rename_axis('geoid')
                       .reset_index().assign(measure='Competition'))
                res.append(pd.concat([xs, fca]).assign(
                    scale=scale, mode=mode, purpose=purp,
                    kind=kind, thresh=str(thresh)))
    xs = pd.concat(res).reset_index(drop=1)
    catCols = 'measure purpose kind mode thresh scale geoid'.split()
    xs = xs.astype({x: CAT for x in catCols} | D(access=F32))
    xs = xs[catCols + ['access']]
    xs.to_parquet(U.mkfile(outpath))
    return xs

# xs = get_access('msa/birmingham', overwrite=1); xs # 58s
# xs = get_access('indiana', tt_in, overwrite=1); xs # 3m48s

### 2.3.1. Indiana (GDM)

In [11]:
xs_in = get_access('indiana', tt_in, overwrite=0).disp() # 3m18s

3,166,152 rows x 8 cols; Memory: 37.0 MiB


,measure,purpose,kind,mode,thresh,scale,geoid,access
,<category>,<category>,<category>,<category>,<category>,<category>,<category>,<float32>
0,Contour,Work,All,Bike,15,BG,180010301001,90.0


### 2.3.2. Top MSAs (OSRM)

In [12]:
def get_access_msa(msa):
    times = get_tt_msa(msa)
    return get_access(f'msa/{msa}', times)

msas = os.listdir(f'{DATA}/access/distances/osrm/msa')
# %time pqdm(msas, get_access_msa, n_jobs=25); # 2h30m2s

## 3.1. Maximum accessibility

In [13]:
max_speeds = {k: v * U.MPH2MPS for k, v in D(
    Drive=70, Walk=3.1, Bike=16, Transit=20).items()}
max_time = 60 * 60 # maximum allowed time (seconds)

In [14]:
def get_max_tt(query, max_speeds=max_speeds, max_time=max_time):
    zones = U.load(DATA / 'zones/zones_2020.parquet',
                   filters=[(k, '==', v) for k, v in query.items()],
                   columns=['geoid', 'scale', 'geometry'])
    xy = zones.centroid.get_coordinates() # columns=['x','y']
    allZones = pd.concat([zones[['geoid', 'scale']], xy], axis=1)
    res = []
    for scale, zones in allZones.groupby('scale'):
        zones = zones.reset_index(drop=1)
        tree = KDTree(zones[['x', 'y']])
        for mode, maxSpeed in max_speeds.items():
            maxDist = max_time * maxSpeed
            src, trg = [zones.loc[idx].reset_index(drop=1) for idx in 
                        tree.query_pairs(maxDist, output_type='ndarray').T]
            dist = ((src.x - trg.x) ** 2 + (src.y - trg.y) ** 2) ** 0.5
            time = dist / maxSpeed
            src, trg = src.geoid.values, trg.geoid.values
            nodes = list(set(src) | set(trg))
            times = pd.concat([Pdf(D(src=nodes, trg=nodes, time=0)),
                               Pdf(D(src=src, trg=trg, time=time)),
                               Pdf(D(src=trg, trg=src, time=time))])
            res.append(times.assign(scale=scale, mode=mode))
    times = pd.concat(res).reset_index(drop=1).astype(D(time=F32))
    times.time = (times.time / 60).astype(F32) # convert to minutes
    catCols = ['scale', 'mode', 'src', 'trg']
    times = times[catCols + ['time']].astype({x: CAT for x in catCols})
    return times

tt_max_in = get_max_tt(D(state='IN')).disp() # 19s

9,821,504 rows x 5 cols; Memory: 95.1 MiB


,scale,mode,src,trg,time
,<category>,<category>,<category>,<category>,<float32>
0,BG,Drive,180319693003,180319693003,0.0


### 3.1.1. Indiana

In [15]:
xs_max_in = get_access('indiana_max', tt_max_in, overwrite=0).disp() # 6m19s

3,496,399 rows x 8 cols; Memory: 40.7 MiB


,measure,purpose,kind,mode,thresh,scale,geoid,access
,<category>,<category>,<category>,<category>,<category>,<category>,<category>,<float32>
0,Contour,Work,All,Bike,15,BG,180010301001,90.0


### 3.1.2. Top MSAs

In [16]:
def get_max_access_msa(msa):
    try:
        outpath = DATA / f'access/access/msa_max/{msa}.parquet'
        if outpath.exists(): return
        name = msa.replace('-', ' ').title()
        times = get_max_tt(D(msa=name))
        return get_access(f'msa_max/{msa}', times)
    except Exception as e:
        print('ERROR:', msa, e)

msas = sorted(os.listdir(f'{DATA}/access/distances/osrm/msa'))
# x = get_max_access_msa('austin'); x # 2m39s
print(f'Started: {dt.datetime.now()}')
%time pqdm(msas, get_max_access_msa, n_jobs=2) # 1h5m39s + 3h42m33s
print(f'Ended: {dt.datetime.now()}')

Started: 2024-04-04 18:27:30.705084


QUEUEING TASKS | :   0%|          | 0/50 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/50 [00:00<?, ?it/s]

ERROR: birmingham No objects to concatenate
ERROR: fresno No objects to concatenate


  0%|          | 0/5 [00:00<?, ?it/s]

...

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

ERROR: st-louis No objects to concatenate


COLLECTING RESULTS | :   0%|          | 0/50 [00:00<?, ?it/s]

CPU times: user 734 ms, sys: 332 ms, total: 1.07 s
Wall time: 3h 42min 33s
Ended: 2024-04-04 22:10:04.473561


## 2.4. Export for publication

In [13]:
# def export_access_msa1(path):
#     df = U.load(path, columns='measure kind mode thresh geoid access'.split())
#     df.measure = df.measure.astype(str).map(
#         D(Contour='C', Gravity='G', Competition='F'))
#     df.measure += (df['mode'].str[0] + df['thresh'].astype(str))
#     df.kind = df.kind.astype(str).map({v: k for k, v in D(
#         J0='All', JE='Low edu', JR='POC', JW='Low wage',
#         P0='Total', PE='Education', PG='Groceries',
#         PM='Medical', PS='Social Support').items()})
#     combs = [x + y for x, y in it.product(
#         'CGF', 'D15 D30 D45 D60 W15 W30 B15 B30'.split())]
#     df = df[df.measure.isin(combs)]
#     df.measure += df.kind
#     df = df.pivot_table('access', 'geoid', 'measure')
#     df = df.fillna(0).astype(F32)
#     return df

# # x = export_access_msa1(DATA / 'access/base/msa/austin.parquet'); x

In [14]:
# 39s
# paths = list((DATA / 'access/base/msa').glob('*.parquet'))
# xsMSApub = pd.concat(pqdm(paths, export_access_msa1, n_jobs=40))
# xsMSApub = (zoneMSA.drop(columns='aland').merge(xsMSApub, on='geoid')
#             .sort_values(['scale', 'geoid']).reset_index(drop=1)
#             .rename(columns=D(geoid='fips', msa='city'))
#             .rename(columns=str.upper)).disp()
# xsMSApub.to_csv(DATA / 'access/export_us_msa_access.csv', index=False)
# xsMSApub = pd.read_csv(DATA / 'access/export_us_msa_access.csv').disp()